<a href="https://colab.research.google.com/github/Rohit1217/Flow/blob/main/Flow_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install matplotlib

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
Device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)

data=trainset.data
data=data.to(dtype=torch.float32)
norm = transforms.Normalize((128,),(128,))
data=norm(data)
data_target=data*0.5+0.5
data=data.view(60000,784)
data=data.to(dtype=torch.float32)

data_target=data_target.view(60000,784)
trainset=TensorDataset(data,data_target)
trainloader=DataLoader(trainset,batch_size=256,shuffle=True)


In [5]:
def gaussian_pdf(x, mean, std_dev):
    return 1 / (torch.sqrt(2 * torch.tensor(np.pi)) * std_dev) * torch.exp(-0.5 * ((x - mean) / std_dev)**2)


In [6]:
class AffineCoupling(nn.Module):
  def __init__(self,input_layer):
    super(AffineCoupling,self).__init__()
    self.inp=input_layer//2
    self.fc1=nn.Linear(self.inp,1000).to(Device)
    self.fc2=nn.Linear(1000,1000).to(Device)
    self.fc3=nn.Linear(1000,1000).to(Device)
    self.fc4=nn.Linear(1000,1000).to(Device)
    self.fc5=nn.Linear(1000,self.inp).to(Device)
    #self.fc6=nn.Linear(1000,self.inp).to(Device)
    self.bn1=nn.BatchNorm1d(1000).to(Device)
    self.bn2=nn.BatchNorm1d(1000).to(Device)
    self.bn3=nn.BatchNorm1d(1000).to(Device)
    self.bn4=nn.BatchNorm1d(1000).to(Device)
    self.bn5=nn.BatchNorm1d(self.inp).to(Device)


  def coupling_shuffler(self,x):
    b,c=x.shape
    layer=torch.rand(b,c).to(Device)
    for i in range(c):
      if i%2==0:
        layer[:,c//2+i//2]=x[:,i]
      else:
        layer[:,i//2]=x[:,i]
    return layer

  def inverse(self,x):
    b,c=x.shape
    x0=x[:,:c//2]
    x1=x[:,c//2:]
    z=F.relu(self.bn1(self.fc1(x0)))
    z=F.relu(self.bn2(self.fc2(z)))
    z=F.relu(self.bn3(self.fc3(z)))
    z=F.relu(self.bn4(self.fc4(z)))
    z=F.relu(self.bn5(self.fc5(z)))
    x1=x1-z
    x=torch.cat((x0, x1), 1)
    layer=torch.rand(b,c).to(Device)
    for i in range(c):
      if i>=c//2:
        layer[:,2*i-c]=x[:,i]
      else:
        layer[:,i*2+1]=x[:,i]
    return layer


  def forward(self,x):
    b,c=x.shape
    x=self.coupling_shuffler(x)
    x0=x[:,:c//2]
    x1=x[:,c//2:]

    z=F.relu(self.bn1(self.fc1(x0)))
    z=F.relu(self.bn2(self.fc2(z)))
    z=F.relu(self.bn3(self.fc3(z)))
    z=F.relu(self.bn4(self.fc4(z)))
    z=F.relu(self.bn5(self.fc5(z)))

    x1=x1+z
    x=torch.cat((x0, x1), 1)
    return x

x=torch.randn(2,20)
print(x)
aff=AffineCoupling(20)
y=aff(x)
print(aff(x))
aff.inverse(y)


tensor([[-1.7312, -0.2813, -2.1592,  1.4464,  1.8657,  1.9125,  0.1874, -0.1299,
          0.7409, -0.7276, -0.1223,  0.6982, -0.8321, -0.5096,  0.1882, -0.7245,
         -1.0169,  0.6446,  0.2254,  2.6652],
        [-0.4860,  0.7086,  0.5798, -0.1686, -1.1664, -0.1599, -1.9507, -1.3662,
          1.7909, -0.9496, -0.5812,  0.3116, -0.3924, -0.6751,  1.2415,  0.8271,
          0.6359, -0.0915, -0.3848,  0.6819]])
tensor([[-0.2813,  1.4464,  1.9125, -0.1299, -0.7276,  0.6982, -0.5096, -0.7245,
          0.6446,  2.6652, -0.7313, -2.1592,  2.8656,  1.1873,  1.7409,  0.8775,
         -0.8321,  0.1882, -0.0175,  1.2249],
        [ 0.7086, -0.1686, -0.1599, -1.3662, -0.9496,  0.3116, -0.6751,  0.8271,
         -0.0915,  0.6819, -0.4860,  1.5797, -1.1664, -1.9507,  1.7909, -0.5812,
          0.5953,  2.2414,  0.6359, -0.3848]], device='cuda:0',
       grad_fn=<CatBackward0>)


tensor([[-1.7312, -0.2813, -2.1592,  1.4464,  1.8657,  1.9125,  0.1874, -0.1299,
          0.7409, -0.7276, -0.1223,  0.6982, -0.8321, -0.5096,  0.1882, -0.7245,
         -1.0169,  0.6446,  0.2254,  2.6652],
        [-0.4860,  0.7086,  0.5798, -0.1686, -1.1664, -0.1599, -1.9507, -1.3662,
          1.7909, -0.9496, -0.5812,  0.3116, -0.3924, -0.6751,  1.2415,  0.8271,
          0.6359, -0.0915, -0.3848,  0.6819]], device='cuda:0',
       grad_fn=<CopySlices>)

In [7]:
class Rescale(nn.Module):
    def __init__(self, input_layer):
        super(Rescale, self).__init__()
        self.weight = nn.Parameter(torch.ones(input_layer)).to(Device)

    def forward(self, x):
        x = torch.exp(self.weight) * x
        y=self.weight+x-x
        return x,y

In [8]:
class Flow(nn.Module):
  def __init__(self,input_layer):
    super(Flow,self).__init__()
    self.inp=input_layer
    self.ac1=AffineCoupling(input_layer)
    self.ac2=AffineCoupling(input_layer)
    self.ac3=AffineCoupling(input_layer)
    self.ac4=AffineCoupling(input_layer)
    self.ac5=AffineCoupling(input_layer)
    self.ac6=AffineCoupling(input_layer)
    self.scaling=Rescale(input_layer)

  def inverse(self,x):
    with torch.no_grad():
      scaling_diag=self.scaling.weight
      x = torch.matmul(x, torch.diag(torch.reciprocal(torch.exp(scaling_diag))))
      x=self.ac5.inverse(x)
      x=self.ac6.inverse(x)
      x=self.ac4.inverse(x)
      x=self.ac3.inverse(x)
      x=self.ac2.inverse(x)
      x=self.ac1.inverse(x)
      return x


  def forward(self,x):
    x=self.ac1(x)
    x=self.ac2(x)
    x=self.ac3(x)
    x=self.ac4(x)
    x=self.ac5(x)
    x=self.ac6(x)
    x,y= self.scaling(x)
    return x,y

x=torch.rand(2,6)
x=x.to(Device)
y1=x
flows=Flow(6)
y,s=flows(x)
x=flows.inverse(y)
print(y1,x,y,s)


tensor([[0.2013, 0.7807, 0.1169, 0.9708, 0.6213, 0.3164],
        [0.1446, 0.3916, 0.1905, 0.8455, 0.0196, 0.8802]], device='cuda:0') tensor([[ 0.1838,  0.7806,  0.1168, -0.0292,  0.6213, -0.6745],
        [ 0.1440,  0.3912,  0.1905,  1.8445,  0.0198,  1.8802]],
       device='cuda:0') tensor([[3.2056, 4.8401, 8.4718, 2.6388, 7.1249, 9.0132],
        [3.1093, 3.7814, 3.2360, 7.7320, 5.4892, 5.1109]], device='cuda:0',
       grad_fn=<MulBackward0>) tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]], device='cuda:0',
       grad_fn=<SubBackward0>)


In [17]:
def loss_f(x,diag):
  #x=0.5*(torch.pow(x,2))+0.5*torch.log(torch.tensor(2*np.pi))
  #torch.sum(diag)
  #print(torch.mean(-(diag)+(0.5*(torch.pow(x,2))+0.5*torch.log(torch.tensor(2*np.pi)))))
  return torch.mean(-torch.sum(diag)+torch.sum(0.5*(torch.pow(x,2))+0.5*torch.log(torch.tensor(2*np.pi)),dim=1))

flow=Flow(784)
x=torch.randn(2,784)
y,s=flow(x)
print(y,s)
loss_f(y,s)

tensor([[-1.3630, -2.6777, -4.6162,  ...,  8.5511, 11.0031,  9.0026],
        [-4.8333, -0.2578,  0.5499,  ...,  6.2304,  8.0249,  2.8552]],
       device='cuda:0', grad_fn=<MulBackward0>) tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
       device='cuda:0', grad_fn=<SubBackward0>)


tensor(11490.6543, device='cuda:0', grad_fn=<MeanBackward0>)

In [18]:
epochs=1000
model=Flow(784)
optimizer=optim.Adam(model.parameters(),lr=0.001)
model=model.to(Device)

In [ ]:
model.train()
for epoch in range(epochs):
  total_loss=0
  count=0
  for input,_ in trainloader:
    input=input.to(Device)
    input,scaling=model(input)
    loss=loss_f(input,scaling)

    total_loss+=loss.item()
    count+=1

    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

    #print(total_loss)
    #print("Gradient of self.scaling_diag:")
    #print(model.scaling.weight)
    #print(model.scaling.weight)
  print(f'total_loss={total_loss},average_loss={total_loss/count},epoch={epoch}')


total_loss=-46692678.078125,average_loss=-198692.24714095745,epoch=0
total_loss=-46757501.65625,average_loss=-198968.0921542553,epoch=1


In [ ]:
with torch.no_grad():
  #model.eval()
  x=data[:64]
  #print(x)
  #x=torch.randn(64,784)
  x=x.to('cuda:0')
  #x=flow(x)
  x = x.view(8, 8, 28, 28)
  x = x.to('cpu')
  image_np = x.detach().numpy()

  # Create a figure and a set of subplots arranged in an 8x8 grid
  fig, axes = plt.subplots(8, 8, figsize=(8, 8))

  # Iterate through the subplots and display each image
  for i in range(8):
      for j in range(8):
          axes[i, j].imshow(image_np[i, j], cmap='gray')
          axes[i, j].axis('off')  # Turn off axis labels

  plt.show()

In [ ]:
print(model.scaling_diag)

In [ ]:
with torch.no_grad():
  #model.eval()
  x=data[:64]
  #print(x)
  x=torch.randn(64,784)
  x=x.to('cuda:0')
  #x=model(x)

  x=model.inverse(x)
  print(x[0])
  x = torch.clamp(x, min=-1, max=1)
  print(x[0])
  x=x*0.5+0.5
  print(x[0])
  x = x.view(8,8, 28, 28)
  x = x.to('cpu')
  image_np = x.detach().numpy()


  # Create a figure and a set of subplots arranged in an 8x8 grid
  fig, axes = plt.subplots(8, 8, figsize=(8, 8))

  # Iterate through the subplots and display each image
  for i in range(8):
      for j in range(8):
          axes[i, j].imshow(image_np[i, j], cmap='gray')
          axes[i, j].axis('off')  # Turn off axis labels

  plt.show()